In [4]:
from keras.models import load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix


import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
import scipy
from scipy import ndimage

dev_data_dir = "../gajankumar1016/Artify/style_classification/dev"
test_data_dir = "../gajankumar1016/Artify/style_classification/test"
img_width, img_height= 256, 256
nb_dev_samples = 1300
nb_test_samples = 1300
batch_size = 128

d = test_data_dir

# for art_style in os.listdir(d):
#     i = 0
#     subdir = os.path.join(d, art_style)
#     for img in os.listdir(subdir):
#         if i % 20000 == 0:
#             img_path = os.path.join(subdir, img)
#             img = np.array(ndimage.imread(img_path, flatten=False))
#             np_image_arr = scipy.misc.imresize(img, size=(img_width, img_height))
#             plt.imshow(img)
#             plt.show()
            
#         i += 1

# style_dir = os.path.join(dev_data_dir, 'Impressionism')
# i = 0
# for img in os.listdir(style_dir):
#     if i % 20 == 0:
#         img_path = os.path.join(style_dir, img)
#         img = np.array(ndimage.imread(img_path, flatten=False))
#         img = scipy.misc.imresize(img, size=(img_width, img_height))
#         plt.imshow(img)
#         plt.show()
    
#     i += 1
    

In [5]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_generator = test_datagen.flow_from_directory(
    dev_data_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

print(test_generator.class_indices)

model = load_model("VGG_model_weights2.h5")

Found 1300 images belonging to 13 classes.
{'Romanticism': 11, 'Post_Impressionism': 8, 'Northern_Renaissance': 7, 'Cubism': 3, 'Rococo': 10, 'Impressionism': 5, 'Expressionism': 4, 'Naive_Art_Primitivism': 6, 'Art_Nouveau_Modern': 1, 'Abstract_Expressionism': 0, 'Baroque': 2, 'Symbolism': 12, 'Realism': 9}


In [7]:
print(model.metrics_names)
model.evaluate_generator(generator=test_generator, steps=nb_test_samples//(batch_size), verbose=1)

['loss', 'acc']
10/10 [==============================] - 34s 3s/step


[2.0059885501861574, 0.4953125]

In [13]:
#Confusion Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, nb_dev_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [15]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = [style for style, idx in sorted(test_generator.class_indices.items(), key=lambda t : t[1])]
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Confusion Matrix
[[14  5 14  3  6  3  7 14  5  8  7  8  6]
 [ 6  7  9  5 10  6  8 11 10  7  8 10  3]
 [ 9  5  7 10  9  8  7  8  8  8  6  5 10]
 [ 9  9 11  8  8  2  7  8  5  9  7 10  7]
 [ 7  8 10  6  8  3 11  8  7  9  4 14  5]
 [ 9 10  8  3 10  5 11  7  9 11  8  5  4]
 [12  6  9  7 10 10  5  9  3  9  8  7  5]
 [ 9  8  3  8 12  6 13 13  4 11  2  6  5]
 [ 7  4  9  8  7  4 12  5  5  6 10  9 14]
 [ 8  5  8  2 12  2 13 12  4 10  7 11  6]
 [ 9  3  5  7  6 10 13 15  6  5  1  8 12]
 [ 2  7 10  9  8  7  7  9  7 12  5  9  8]
 [ 6  8  7  8  9  8  7  8  4  7 12 12  4]]
Classification Report
                        precision    recall  f1-score   support

Abstract_Expressionism       0.13      0.14      0.14       100
    Art_Nouveau_Modern       0.08      0.07      0.08       100
               Baroque       0.06      0.07      0.07       100
                Cubism       0.10      0.08      0.09       100
         Expressionism       0.07      0.08      0.07       100
         Impressionism       

In [12]:
# def get_style_classes(root_dir):
#     return [d for d in os.listdir(root_dir)]
        
# style_classes = get_style_classes(dev_data_dir)
# print(style_classes)